In [1]:
import torch.utils.data as utils
import torch.nn.functional as F
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import numpy as np
import pandas as pd
import math
import time

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline  

In [3]:
print(torch.__version__)

1.2.0+cpu


In [2]:
def PrepareDataset(sparse_mat, dense_mat, BATCH_SIZE = 40, time_lags = np.array([1, 2, 288]), pred_len = 1, valid_len = 2882, test_len = 1441):
    """ Prepare training and testing datasets and dataloaders.
    
    Convert speed/volume/occupancy matrix to training and testing dataset. 
    The vertical axis of speed_matrix is the time axis and the horizontal axis 
    is the spatial axis.
    
    Args:
        speed_matrix: a Matrix containing spatial-temporal speed data for a network
        seq_len: length of input sequence
        pred_len: length of predicted sequence
    Returns:
        Training dataloader
        Testing dataloader
    """
    time_len = sparse_mat.shape[0]
    
    #speed_matrix = speed_matrix.clip(0, 100)
    
    max_speed = np.max(sparse_mat)
    sparse_mat =  sparse_mat / max_speed
    dense_mat = dense_mat / max_speed
    max_lag = np.max(time_lags)
    speed_sequences, speed_sequences_dense, speed_labels = [], [], []
    for i in range(time_len - max_lag - pred_len + 1):
        speed_sequences.append(sparse_mat[i+time_lags-1, :])
        speed_sequences_dense.append(dense_mat[i+time_lags-1, :])
        speed_labels.append(dense_mat[i+max_lag:i+max_lag+pred_len, :])
    speed_sequences, speed_labels = np.asarray(speed_sequences), np.asarray(speed_labels)
    sample_size = speed_sequences.shape[0]
    
#     test_len = int(sample_size * test_proportion)
#     valid_len = int(sample_size * valid_proportion)
    train_index = sample_size - test_len - valid_len
    valid_index = train_index + valid_len
    
    train_data, train_label = speed_sequences[:train_index], speed_labels[:train_index]
    valid_data, valid_label = speed_sequences[train_index:valid_index], speed_labels[train_index:valid_index]
    test_data, test_label = speed_sequences[valid_index:], speed_labels[valid_index:]
    
    train_data, train_label = torch.Tensor(train_data), torch.Tensor(train_label)
    valid_data, valid_label = torch.Tensor(valid_data), torch.Tensor(valid_label)
    test_data, test_label = torch.Tensor(test_data), torch.Tensor(test_label)

    train_dataset = utils.TensorDataset(train_data, train_label)
    valid_dataset = utils.TensorDataset(valid_data, valid_label)
    test_dataset = utils.TensorDataset(test_data, test_label)
    print()
    print('Test_set shape:')
    print(test_data.shape)
    train_dataloader = utils.DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle=True, drop_last = True)
    valid_dataloader = utils.DataLoader(valid_dataset, batch_size = BATCH_SIZE, shuffle=True, drop_last = True)
    test_dataloader = utils.DataLoader(test_dataset, batch_size = test_data.shape[0], shuffle=True, drop_last = True)
    return train_dataloader, valid_dataloader, test_dataloader, max_speed

In [3]:
def TrainModel(model, train_dataloader, valid_dataloader, learning_rate = 1e-5, num_epochs = 300, patience = 10, min_delta = 0.00001):
    
    inputs, labels = next(iter(train_dataloader))
    [batch_size, step_size, fea_size] = inputs.size()
    input_dim = fea_size
    hidden_dim = fea_size
    output_dim = fea_size
    
    #model.cuda()
    
    loss_MSE = torch.nn.MSELoss()
    loss_L1 = torch.nn.L1Loss()

    learning_rate = 1e-5
    optimizer = torch.optim.RMSprop(model.parameters(), lr = learning_rate)
    
    use_gpu = torch.cuda.is_available()
    
    interval = 100
    losses_train = []
    losses_valid = []
    losses_epochs_train = []
    losses_epochs_valid = []
    
    cur_time = time.time()
    pre_time = time.time()
    
    # Variables for Early Stopping
    is_best_model = 0
    patient_epoch = 0
    
    for epoch in range(num_epochs):
#         print('Epoch {}/{}'.format(epoch, num_epochs - 1))
#         print('-' * 10)
        
        trained_number = 0
        
        valid_dataloader_iter = iter(valid_dataloader)
        
        losses_epoch_train = []
        losses_epoch_valid = []

        for data in train_dataloader:
            inputs, labels = data

            if inputs.shape[0] != batch_size:
                continue

            if use_gpu:
                inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
            else: 
                inputs, labels = Variable(inputs), Variable(labels)
                
            model.zero_grad()

            outputs = model(inputs)
            
            loss_train = loss_MSE(outputs, torch.squeeze(labels))
            
            losses_train.append(loss_train.data)
            losses_epoch_train.append(loss_train.data)
            
            optimizer.zero_grad()
            
            loss_train.backward()
            
            optimizer.step()
            
            # validation 
            try: 
                inputs_val, labels_val = next(valid_dataloader_iter)
            except StopIteration:
                valid_dataloader_iter = iter(valid_dataloader)
                inputs_val, labels_val = next(valid_dataloader_iter)
            
            if use_gpu:
                inputs_val, labels_val = Variable(inputs_val.cuda()), Variable(labels_val.cuda())
            else: 
                inputs_val, labels_val = Variable(inputs_val), Variable(labels_val)

            outputs_val= model(inputs_val)

            loss_valid = loss_MSE(outputs_val, torch.squeeze(labels_val))
            losses_valid.append(loss_valid.data)
            losses_epoch_valid.append(loss_valid.data)
            
            # output
            trained_number += 1
            
        avg_losses_epoch_train = sum(losses_epoch_train) / float(len(losses_epoch_train))
        avg_losses_epoch_valid = sum(losses_epoch_valid) / float(len(losses_epoch_valid))
        losses_epochs_train.append(avg_losses_epoch_train)
        losses_epochs_valid.append(avg_losses_epoch_valid)
        
        # Early Stopping
        if epoch == 0:
            is_best_model = 1
            best_model = model
            min_loss_epoch_valid = 10000.0
            if avg_losses_epoch_valid < min_loss_epoch_valid:
                min_loss_epoch_valid = avg_losses_epoch_valid
        else:
            if min_loss_epoch_valid - avg_losses_epoch_valid > min_delta:
                is_best_model = 1
                best_model = model
                min_loss_epoch_valid = avg_losses_epoch_valid 
                patient_epoch = 0
            else:
                is_best_model = 0
                patient_epoch += 1
                if patient_epoch >= patience:
                    print('Early Stopped at Epoch:', epoch)
                    break
        
        # Print training parameters
        cur_time = time.time()
        print('Epoch: {}, train_loss: {}, valid_loss: {}, time: {}, best model: {}'.format( \
                    epoch, \
                    np.around(avg_losses_epoch_train, decimals=8),\
                    np.around(avg_losses_epoch_valid, decimals=8),\
                    np.around([cur_time - pre_time] , decimals=2),\
                    is_best_model) )
        pre_time = cur_time
    return best_model, [losses_train, losses_valid, losses_epochs_train, losses_epochs_valid]


In [4]:
def TestModel(model, test_dataloader, max_speed):
    
    inputs, labels = next(iter(test_dataloader))
    [batch_size, step_size, fea_size] = inputs.size()

    cur_time = time.time()
    pre_time = time.time()
    
    use_gpu = torch.cuda.is_available()
    
    loss_MSE = torch.nn.MSELoss()
    loss_L1 = torch.nn.MSELoss()
    
    tested_batch = 0
    
    losses_mse = []
    losses_l1 = [] 
    MAPE = []
    for data in test_dataloader:
        
        inputs, labels = data
        
        if inputs.shape[0] != batch_size:
            continue
    
        if use_gpu:
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        else: 
            inputs, labels = Variable(inputs), Variable(labels)

        # rnn.loop() 
        hidden = model.initHidden(batch_size)

        outputs = None
        outputs = model(inputs)
        outputs_np = np.array(outputs.data.numpy())
        
        sqlabel = torch.squeeze(labels)
        loss_MSE = torch.nn.MSELoss()
        loss_L1 = torch.nn.L1Loss()
        loss_mse = loss_MSE(outputs, sqlabel)
        loss_l1 = loss_L1(outputs, sqlabel)
        loss_MAPE = torch.mean(torch.abs(outputs[sqlabel != 0] - sqlabel[sqlabel != 0]) / sqlabel[sqlabel != 0])
        
        losses_mse.append(loss_mse.cpu().data.numpy())
        losses_l1.append(loss_l1.cpu().data.numpy())
        MAPE.append(loss_MAPE.cpu().data.numpy())
        
        tested_batch += 1
    
        if tested_batch % 1000 == 0:
            cur_time = time.time()
            print('Tested #: {}, loss_l1: {}, loss_mse: {}, time: {}'.format( \
                  tested_batch * batch_size, \
                  np.around([loss_l1.data[0]], decimals=8), \
                  np.around([loss_mse.data[0]], decimals=8), \
                  np.around([cur_time - pre_time], decimals=8) ) )
            pre_time = cur_time
    losses_l1 = np.array(losses_l1)
    losses_mse = np.array(losses_mse)
    MAPE = np.array(MAPE)
    mean_l1 = np.mean(losses_l1) * max_speed
    std_l1 = np.std(losses_l1) * max_speed
    RMSE_ = np.sqrt(np.mean(losses_mse)) * max_speed
    MAPE_ = np.mean(MAPE) * 100
    print('Tested: L1_mean: {}, L1_std : {}, MAPE : {}, RMSE : {}'.format(mean_l1, std_l1, MAPE_, RMSE_))
    return [losses_l1, losses_mse, mean_l1, std_l1, outputs_np]

In [5]:
class LSTM(nn.Module):
    def __init__(self, input_size, cell_size, hidden_size, output_last = True):
        """
        cell_size is the size of cell_state.
        hidden_size is the size of hidden_state, or say the output_state of each step
        """
        super(LSTM, self).__init__()
        
        self.cell_size = cell_size
        self.hidden_size = hidden_size
        self.fl = nn.Linear(input_size + hidden_size, hidden_size)
        self.il = nn.Linear(input_size + hidden_size, hidden_size)
        self.ol = nn.Linear(input_size + hidden_size, hidden_size)
        self.Cl = nn.Linear(input_size + hidden_size, hidden_size)
        
        self.output_last = output_last
        
    def step(self, input, Hidden_State, Cell_State):
        combined = torch.cat((input, Hidden_State), 1)
        f = F.sigmoid(self.fl(combined))
        i = F.sigmoid(self.il(combined))
        o = F.sigmoid(self.ol(combined))
        C = F.tanh(self.Cl(combined))
        Cell_State = f * Cell_State + i * C
        Hidden_State = o * F.tanh(Cell_State)
        
        return Hidden_State, Cell_State
    
    def forward(self, inputs):
        batch_size = inputs.size(0)
        time_step = inputs.size(1)
        Hidden_State, Cell_State = self.initHidden(batch_size)
        
        if self.output_last:
            for i in range(time_step):
                Hidden_State, Cell_State = self.step(torch.squeeze(inputs[:,i:i+1,:]), Hidden_State, Cell_State)  
            return Hidden_State
        else:
            outputs = None
            for i in range(time_step):
                Hidden_State, Cell_State = self.step(torch.squeeze(inputs[:,i:i+1,:]), Hidden_State, Cell_State)  
                if outputs is None:
                    outputs = Hidden_State.unsqueeze(1)
                else:
                    outputs = torch.cat((outputs, Hidden_State.unsqueeze(1)), 1)
            return outputs
    
    def initHidden(self, batch_size):
        use_gpu = torch.cuda.is_available()
        if use_gpu:
            Hidden_State = Variable(torch.zeros(batch_size, self.hidden_size).cuda())
            Cell_State = Variable(torch.zeros(batch_size, self.hidden_size).cuda())
            return Hidden_State, Cell_State
        else:
            Hidden_State = Variable(torch.zeros(batch_size, self.hidden_size))
            Cell_State = Variable(torch.zeros(batch_size, self.hidden_size))
            return Hidden_State, Cell_State
        

In [18]:
class ConvLSTM(nn.Module):
    def __init__(self, input_size, cell_size, hidden_size, output_last = True):
        """
        cell_size is the size of cell_state.
        hidden_size is the size of hidden_state, or say the output_state of each step
        """
        super(ConvLSTM, self).__init__()
        
        self.cell_size = cell_size
        self.hidden_size = hidden_size
        self.fl = nn.Linear(input_size + hidden_size, hidden_size)
        self.il = nn.Linear(input_size + hidden_size, hidden_size)
        self.ol = nn.Linear(input_size + hidden_size, hidden_size)
        self.Cl = nn.Linear(input_size + hidden_size, hidden_size)
        
        self.conv = nn.Conv1d(1, hidden_size, hidden_size)
        
        self.output_last = output_last
        
    def step(self, input, Hidden_State, Cell_State):
        
        conv = self.conv(input)
        
        combined = torch.cat((conv, Hidden_State), 1)
        f = F.sigmoid(self.fl(combined))
        i = F.sigmoid(self.il(combined))
        o = F.sigmoid(self.ol(combined))
        C = F.tanh(self.Cl(combined))
        Cell_State = f * Cell_State + i * C
        Hidden_State = o * F.tanh(Cell_State)
        
        return Hidden_State, Cell_State
    
    def forward(self, inputs):
        batch_size = inputs.size(0)
        time_step = inputs.size(1)
        Hidden_State, Cell_State = self.initHidden(batch_size)
        
        if self.output_last:
            for i in range(time_step):
                Hidden_State, Cell_State = self.step(torch.squeeze(inputs[:,i:i+1,:]), Hidden_State, Cell_State)  
            return Hidden_State
        else:
            outputs = None
            for i in range(time_step):
                Hidden_State, Cell_State = self.step(torch.squeeze(inputs[:,i:i+1,:]), Hidden_State, Cell_State)  
                if outputs is None:
                    outputs = Hidden_State.unsqueeze(1)
                else:
                    outputs = torch.cat((outputs, Hidden_State.unsqueeze(1)), 1)
            return outputs
    
    def initHidden(self, batch_size):
        use_gpu = torch.cuda.is_available()
        if use_gpu:
            Hidden_State = Variable(torch.zeros(batch_size, self.hidden_size).cuda())
            Cell_State = Variable(torch.zeros(batch_size, self.hidden_size).cuda())
            return Hidden_State, Cell_State
        else:
            Hidden_State = Variable(torch.zeros(batch_size, self.hidden_size))
            Cell_State = Variable(torch.zeros(batch_size, self.hidden_size))
            return Hidden_State, Cell_State
        

In [19]:
class LocalizedSpectralGraphConvolution(nn.Module):
    def __init__(self, A, K):
        
        super(LocalizedSpectralGraphConvolution, self).__init__()
        
        
        self.K = K
#         self.A = A.cuda()
        self.A = A
        feature_size = A.shape[0]
#         self.D = torch.diag(torch.sum(self.A, dim=0)).cuda()
        self.D = torch.diag(torch.sum(self.A, dim=0))
    
#         I = torch.eye(feature_size,feature_size).cuda()
        I = torch.eye(feature_size,feature_size)
        self.L = I - torch.inverse(torch.sqrt(self.D)).matmul(self.A).matmul(torch.inverse(torch.sqrt(self.D))) 
        
        L_temp = I
        for i in range(K):
            L_temp = torch.matmul(L_temp, self.L)
            if i == 0:
                self.L_tensor = torch.unsqueeze(L_temp, 2)
            else:
                self.L_tensor = torch.cat((self.L_tensor, torch.unsqueeze(L_temp, 2)), 2)
            
#         self.L_tensor = Variable(self.L_tensor.cuda(), requires_grad=False)
        self.L_tensor = Variable(self.L_tensor, requires_grad=False)
#         self.params = Parameter(torch.FloatTensor(K).cuda())
        self.params = Parameter(torch.FloatTensor(K))
        
        stdv = 1. / math.sqrt(K)
        for i in range(K):
            self.params[i].data.uniform_(-stdv, stdv)

    def forward(self, input):
        x = input

        conv = x.matmul( torch.sum(self.params.expand_as(self.L_tensor) * self.L_tensor, 2) )

        return conv
        
        
class LocalizedSpectralGraphConvolutionalLSTM(nn.Module):
    
    def __init__(self, K, A, feature_size, Clamp_A=True, output_last = True):
        '''
        Args:
            K: K-hop graph
            A: adjacency matrix
            FFR: free-flow reachability matrix
            feature_size: the dimension of features
            Clamp_A: Boolean value, clamping all elements of A between 0. to 1.
        '''
        super(LocalizedSpectralGraphConvolutionalLSTM, self).__init__()
        self.feature_size = feature_size
        self.hidden_size = feature_size
        
        self.K = K
        self.A = A
        self.gconv = LocalizedSpectralGraphConvolution(A, K)
    
        hidden_size = self.feature_size
        input_size = self.feature_size + hidden_size

        self.fl = nn.Linear(input_size, hidden_size)
        self.il = nn.Linear(input_size, hidden_size)
        self.ol = nn.Linear(input_size, hidden_size)
        self.Cl = nn.Linear(input_size, hidden_size)
        
        self.output_last = output_last
        
    def step(self, input, Hidden_State, Cell_State):
        
#         conv_sample_start = time.time()  
        conv = F.relu(self.gconv(input))
#         conv_sample_end = time.time()  
#         print('conv_sample:', (conv_sample_end - conv_sample_start))
        combined = torch.cat((conv, Hidden_State), 1)
        f = F.sigmoid(self.fl(combined))
        i = F.sigmoid(self.il(combined))
        o = F.sigmoid(self.ol(combined))
        C = F.tanh(self.Cl(combined))
        Cell_State = f * Cell_State + i * C
        Hidden_State = o * F.tanh(Cell_State)
        
        return Hidden_State, Cell_State
    
    def Bi_torch(self, a):
        a[a < 0] = 0
        a[a > 0] = 1
        return a
    
    def forward(self, inputs):
        batch_size = inputs.size(0)
        time_step = inputs.size(1)
        Hidden_State, Cell_State = self.initHidden(batch_size)
        
        outputs = None
        
        for i in range(time_step):
            Hidden_State, Cell_State = self.step(torch.squeeze(inputs[:,i:i+1,:]), Hidden_State, Cell_State)  

            if outputs is None:
                outputs = Hidden_State.unsqueeze(1)
            else:
                outputs = torch.cat((outputs, Hidden_State.unsqueeze(1)), 1)
#         print(type(outputs))
        
        if self.output_last:
            return outputs[:,-1,:]
        else:
            return outputs
    
    def initHidden(self, batch_size):
        use_gpu = torch.cuda.is_available()
        if use_gpu:
            Hidden_State = Variable(torch.zeros(batch_size, self.hidden_size).cuda())
            Cell_State = Variable(torch.zeros(batch_size, self.hidden_size).cuda())
            return Hidden_State, Cell_State
        else:
            Hidden_State = Variable(torch.zeros(batch_size, self.hidden_size))
            Cell_State = Variable(torch.zeros(batch_size, self.hidden_size))
            return Hidden_State, Cell_State
    def reinitHidden(self, batch_size, Hidden_State_data, Cell_State_data):
        use_gpu = torch.cuda.is_available()
        if use_gpu:
            Hidden_State = Variable(Hidden_State_data.cuda(), requires_grad=True)
            Cell_State = Variable(Cell_State_data.cuda(), requires_grad=True)
            return Hidden_State, Cell_State
        else:
            Hidden_State = Variable(Hidden_State_data, requires_grad=True)
            Cell_State = Variable(Cell_State_data, requires_grad=True)
            return Hidden_State, Cell_State

In [20]:
class SpectralGraphConvolution(nn.Module):
    def __init__(self, A):
        
        super(SpectralGraphConvolution, self).__init__()
        
        feature_size = A.shape[0]
        
        self.A = A
        self.D = torch.diag(torch.sum(self.A, dim=0))
        self.L = self.D - A
#         self.param = Parameter(torch.FloatTensor(feature_size).cuda())
        self.param = Parameter(torch.FloatTensor(feature_size))
        stdv = 1. / math.sqrt(feature_size)
        self.param.data.uniform_(-stdv, stdv)
        
        self.e, self.v = torch.eig(self.L, eigenvectors=True)
        self.vt = torch.t(self.v)
#         self.v = Variable(self.v.cuda(), requires_grad=False)
#         self.vt = Variable(self.vt.cuda(), requires_grad=False)
        self.v = Variable(self.v, requires_grad=False)
        self.vt = Variable(self.vt, requires_grad=False)
        
    def forward(self, input):
        x = input
        conv_sample_start = time.time()  
        conv = x.matmul(self.v.matmul(torch.diag(self.param)).matmul(self.vt))
        conv_sample_end = time.time()  
        #print('conv_sample:', (conv_sample_end - conv_sample_start))
        return conv
        
class SpectralGraphConvolutionalLSTM(nn.Module):
    
    def __init__(self, K, A, feature_size, Clamp_A=True, output_last = True):
        '''
        Args:
            K: K-hop graph
            A: adjacency matrix
            FFR: free-flow reachability matrix
            feature_size: the dimension of features
            Clamp_A: Boolean value, clamping all elements of A between 0. to 1.
        '''
        super(SpectralGraphConvolutionalLSTM, self).__init__()
        self.feature_size = feature_size
        self.hidden_size = feature_size
        
        self.K = K
        self.A = A
        self.gconv = SpectralGraphConvolution(A)
    
        hidden_size = self.feature_size
        input_size = self.feature_size + hidden_size

        self.fl = nn.Linear(input_size, hidden_size)
        self.il = nn.Linear(input_size, hidden_size)
        self.ol = nn.Linear(input_size, hidden_size)
        self.Cl = nn.Linear(input_size, hidden_size)
        
        self.output_last = output_last
        
    def step(self, input, Hidden_State, Cell_State):
        conv_sample_start = time.time()  
        conv = self.gconv(input)
        conv_sample_end = time.time()  
        #print('conv_sample:', (conv_sample_end - conv_sample_start))
        combined = torch.cat((conv, Hidden_State), 1)
        f = F.sigmoid(self.fl(combined))
        i = F.sigmoid(self.il(combined))
        o = F.sigmoid(self.ol(combined))
        C = F.tanh(self.Cl(combined))
        Cell_State = f * Cell_State + i * C
        Hidden_State = o * F.tanh(Cell_State)
        
        return Hidden_State, Cell_State
    
    def Bi_torch(self, a):
        a[a < 0] = 0
        a[a > 0] = 1
        return a
    
    def forward(self, inputs):
        batch_size = inputs.size(0)
        time_step = inputs.size(1)
        Hidden_State, Cell_State = self.initHidden(batch_size)
        
        outputs = None
        
        train_sample_start = time.time()  
        
        for i in range(time_step):
            Hidden_State, Cell_State = self.step(torch.squeeze(inputs[:,i:i+1,:]), Hidden_State, Cell_State)  

            if outputs is None:
                outputs = Hidden_State.unsqueeze(1)
            else:
                outputs = torch.cat((outputs, Hidden_State.unsqueeze(1)), 1)
        
        train_sample_end = time.time()
        #print('train sample:' , (train_sample_end - train_sample_start))
        if self.output_last:
            return outputs[:,-1,:]
        else:
            return outputs
    
    def initHidden(self, batch_size):
        use_gpu = torch.cuda.is_available()
        if use_gpu:
            Hidden_State = Variable(torch.zeros(batch_size, self.hidden_size).cuda())
            Cell_State = Variable(torch.zeros(batch_size, self.hidden_size).cuda())
            return Hidden_State, Cell_State
        else:
            Hidden_State = Variable(torch.zeros(batch_size, self.hidden_size))
            Cell_State = Variable(torch.zeros(batch_size, self.hidden_size))
            return Hidden_State, Cell_State
    def reinitHidden(self, batch_size, Hidden_State_data, Cell_State_data):
        use_gpu = torch.cuda.is_available()
        if use_gpu:
            Hidden_State = Variable(Hidden_State_data.cuda(), requires_grad=True)
            Cell_State = Variable(Cell_State_data.cuda(), requires_grad=True)
            return Hidden_State, Cell_State
        else:
            Hidden_State = Variable(Hidden_State_data, requires_grad=True)
            Cell_State = Variable(Cell_State_data, requires_grad=True)
            return Hidden_State, Cell_State

In [21]:
class FilterLinear(nn.Module):
    def __init__(self, in_features, out_features, filter_square_matrix, bias=True):
        '''
        filter_square_matrix : filter square matrix, whose each elements is 0 or 1.
        '''
        super(FilterLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        
        use_gpu = torch.cuda.is_available()
        self.filter_square_matrix = None
        if use_gpu:
            self.filter_square_matrix = Variable(filter_square_matrix.cuda(), requires_grad=False)
        else:
            self.filter_square_matrix = Variable(filter_square_matrix, requires_grad=False)
        
        self.weight = Parameter(torch.Tensor(out_features, in_features))
        if bias:
            self.bias = Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)
#         print(self.weight.data)
#         print(self.bias.data)

    def forward(self, input):
        return F.linear(input, self.filter_square_matrix.matmul(self.weight), self.bias)

    def __repr__(self):
        return self.__class__.__name__ + '(' \
            + 'in_features=' + str(self.in_features) \
            + ', out_features=' + str(self.out_features) \
            + ', bias=' + str(self.bias is not None) + ')'

In [22]:
class GraphConvolutionalLSTM(nn.Module):
    
    def __init__(self, K, A, FFR, feature_size, Clamp_A=True, output_last = True):
        '''
        Args:
            K: K-hop graph
            A: adjacency matrix
            FFR: free-flow reachability matrix
            feature_size: the dimension of features
            Clamp_A: Boolean value, clamping all elements of A between 0. to 1.
        '''
        super(GraphConvolutionalLSTM, self).__init__()
        self.feature_size = feature_size
        self.hidden_size = feature_size
        
        self.K = K
        
        self.A_list = [] # Adjacency Matrix List
        A = torch.FloatTensor(A)
        A_temp = torch.eye(feature_size,feature_size)
        for i in range(K):
            A_temp = torch.matmul(A_temp, torch.Tensor(A))
            if Clamp_A:
                # confine elements of A 
                A_temp = torch.clamp(A_temp, max = 1.) 
            self.A_list.append(torch.mul(A_temp, torch.Tensor(FFR)))
#             self.A_list.append(A_temp)
        
        # a length adjustable Module List for hosting all graph convolutions
        self.gc_list = nn.ModuleList([FilterLinear(feature_size, feature_size, self.A_list[i], bias=False) for i in range(K)])                  
        
        hidden_size = self.feature_size
        input_size = self.feature_size * K

        self.fl = nn.Linear(input_size + hidden_size, hidden_size)
        self.il = nn.Linear(input_size + hidden_size, hidden_size)
        self.ol = nn.Linear(input_size + hidden_size, hidden_size)
        self.Cl = nn.Linear(input_size + hidden_size, hidden_size)
        
        # initialize the neighbor weight for the cell state
        self.Neighbor_weight = Parameter(torch.FloatTensor(feature_size))
        stdv = 1. / math.sqrt(feature_size)
        self.Neighbor_weight.data.uniform_(-stdv, stdv)
        
        self.output_last = output_last
        
    def step(self, input, Hidden_State, Cell_State):
        
        x = input

        gc = self.gc_list[0](x)
        for i in range(1, self.K):
            gc = torch.cat((gc, self.gc_list[i](x)), 1)
            
        combined = torch.cat((gc, Hidden_State), 1)
        f = F.sigmoid(self.fl(combined))
        i = F.sigmoid(self.il(combined))
        o = F.sigmoid(self.ol(combined))
        C = F.tanh(self.Cl(combined))

#         NC = torch.mul(Cell_State,  torch.mv(Variable(self.A_list[-1], requires_grad=False).cuda(), self.Neighbor_weight))
        NC = torch.mul(Cell_State,  torch.mv(Variable(self.A_list[-1], requires_grad=False), self.Neighbor_weight))
        Cell_State = f * NC + i * C
        Hidden_State = o * F.tanh(Cell_State)

        return Hidden_State, Cell_State, gc
    
    def Bi_torch(self, a):
        a[a < 0] = 0
        a[a > 0] = 1
        return a
    
    def forward(self, inputs):
        batch_size = inputs.size(0)
        time_step = inputs.size(1)
        Hidden_State, Cell_State = self.initHidden(batch_size)
        
        outputs = None
        
        for i in range(time_step):
            Hidden_State, Cell_State, gc = self.step(torch.squeeze(inputs[:,i:i+1,:]), Hidden_State, Cell_State)  

            if outputs is None:
                outputs = Hidden_State.unsqueeze(1)
            else:
                outputs = torch.cat((outputs, Hidden_State.unsqueeze(1)), 1)
        
        if self.output_last:
            return outputs[:,-1,:]
        else:
            return outputs
    
    def initHidden(self, batch_size):
        use_gpu = torch.cuda.is_available()
        if use_gpu:
            Hidden_State = Variable(torch.zeros(batch_size, self.hidden_size).cuda())
            Cell_State = Variable(torch.zeros(batch_size, self.hidden_size).cuda())
            return Hidden_State, Cell_State
        else:
            Hidden_State = Variable(torch.zeros(batch_size, self.hidden_size))
            Cell_State = Variable(torch.zeros(batch_size, self.hidden_size))
            return Hidden_State, Cell_State
    def reinitHidden(self, batch_size, Hidden_State_data, Cell_State_data):
        use_gpu = torch.cuda.is_available()
        if use_gpu:
            Hidden_State = Variable(Hidden_State_data.cuda(), requires_grad=True)
            Cell_State = Variable(Cell_State_data.cuda(), requires_grad=True)
            return Hidden_State, Cell_State
        else:
            Hidden_State = Variable(Hidden_State_data, requires_grad=True)
            Cell_State = Variable(Cell_State_data, requires_grad=True)
            return Hidden_State, Cell_State

In [31]:
directory = '../datasets/Seattle_loop-data-set/'
A = np.load( directory + 'Loop_Seattle_2015_A.npy')
FFR_5min = np.load( directory + 'Loop_Seattle_2015_reachability_free_flow_5min.npy')
FFR_10min = np.load( directory + 'Loop_Seattle_2015_reachability_free_flow_10min.npy')
FFR_15min = np.load( directory + 'Loop_Seattle_2015_reachability_free_flow_15min.npy')
FFR_20min = np.load( directory + 'Loop_Seattle_2015_reachability_free_flow_20min.npy')
FFR_25min = np.load( directory + 'Loop_Seattle_2015_reachability_free_flow_25min.npy')
FFR = [FFR_5min, FFR_10min, FFR_15min, FFR_20min, FFR_25min]

directory = '../datasets/Seattle_loop-data-set/'
A = np.load( directory + 'Loop_Seattle_2015_A.npy')
dense_mat = np.load( directory + 'dense_mat.npy')

print('Dataset shape:')
print(dense_mat.shape)

missing_rate = 0.4
# =============================================================================
### Random missing (PM) scenario
### Set the PM scenario by:
# rm_random_mat = np.load(directory + 'rm_random_mat.npy')
# binary_mat = np.round(rm_random_mat + 0.5 - missing_rate)
# =============================================================================
# =============================================================================
### Non-random missing (CM) scenario
### Set the CM scenario by:
nm_random_mat = np.load(directory + 'nm_random_mat.npy')
binary_tensor = np.zeros((dense_mat.shape[0], 61, 288))
for i1 in range(binary_tensor.shape[0]):
    for i2 in range(binary_tensor.shape[1]):
        binary_tensor[i1, i2, :] = np.round(nm_random_mat[i1, i2] + 0.5 - missing_rate)
binary_mat = binary_tensor.reshape([binary_tensor.shape[0], binary_tensor.shape[1] * binary_tensor.shape[2]])
# =============================================================================

sparse_mat = np.multiply(dense_mat, binary_mat)
sparse_mat = sparse_mat.T
dense_mat = dense_mat.T

time_lags = np.array([1, 2, 288])
# time_lags = np.arange(1, 11, 1)

train_dataloader, valid_dataloader, test_dataloader, max_speed = PrepareDataset(sparse_mat, dense_mat, BATCH_SIZE = 100, time_lags = time_lags, pred_len = 1, valid_len = 2882, test_len = 1441)

inputs, labels = next(iter(train_dataloader))
[batch_size, step_size, fea_size] = inputs.size()
input_dim = fea_size
hidden_dim = 60
output_dim = fea_size

Dataset shape:
(323, 17568)

Test_set shape:
torch.Size([1441, 3, 323])


In [32]:
lstm = LSTM(input_dim, hidden_dim, output_dim, output_last = True)
lstm, lstm_loss = TrainModel(lstm, train_dataloader, valid_dataloader, num_epochs = 200)
lstm_test = TestModel(lstm, test_dataloader, max_speed )

Epoch: 0, train_loss: 0.3339574337005615, valid_loss: 0.35196104645729065, time: [4.46], best model: 1
Epoch: 1, train_loss: 0.12868650257587433, valid_loss: 0.13059088587760925, time: [4.39], best model: 1
Epoch: 2, train_loss: 0.045536261051893234, valid_loss: 0.043487418442964554, time: [4.49], best model: 1
Epoch: 3, train_loss: 0.02455797977745533, valid_loss: 0.021918289363384247, time: [4.61], best model: 1
Epoch: 4, train_loss: 0.019868889823555946, valid_loss: 0.017253689467906952, time: [4.64], best model: 1
Epoch: 5, train_loss: 0.018493790179491043, valid_loss: 0.016020409762859344, time: [4.75], best model: 1
Epoch: 6, train_loss: 0.017773659899830818, valid_loss: 0.015560319647192955, time: [4.79], best model: 1
Epoch: 7, train_loss: 0.017115440219640732, valid_loss: 0.014991720207035542, time: [4.81], best model: 1
Epoch: 8, train_loss: 0.016383999958634377, valid_loss: 0.014761759899556637, time: [4.84], best model: 1
Epoch: 9, train_loss: 0.015625, valid_loss: 0.014461

Epoch: 78, train_loss: 0.005529860034584999, valid_loss: 0.008864779956638813, time: [6.25], best model: 0
Epoch: 79, train_loss: 0.0055082100443542, valid_loss: 0.008838980458676815, time: [6.25], best model: 0
Epoch: 80, train_loss: 0.005485610105097294, valid_loss: 0.0088197598233819, time: [6.3], best model: 0
Epoch: 81, train_loss: 0.005463730078190565, valid_loss: 0.00883105956017971, time: [6.29], best model: 0
Epoch: 82, train_loss: 0.005444800015538931, valid_loss: 0.008852969855070114, time: [6.21], best model: 0
Early Stopped at Epoch: 83
Tested: L1_mean: 5.005535162566561, L1_std : 0.0, MAPE : 12.14158982038498, RMSE : 7.414167097375825


In [17]:
K = 64
Clamp_A = False
lsgclstm = LocalizedSpectralGraphConvolutionalLSTM(K, torch.Tensor(A), A.shape[0], Clamp_A=Clamp_A, output_last = True)
lsgclstm, lsgclstm_loss = TrainModel(lsgclstm, train_dataloader, valid_dataloader, num_epochs = 200)
lsgclstm_test = TestModel(lsgclstm, test_dataloader, max_speed )

Epoch: 0, train_loss: 0.6803808212280273, valid_loss: 0.7129841446876526, time: [24.28], best model: 1
Epoch: 1, train_loss: 0.6804695129394531, valid_loss: 0.7124403715133667, time: [22.89], best model: 1
Epoch: 2, train_loss: 0.6800757646560669, valid_loss: 0.7127607464790344, time: [22.44], best model: 0
Epoch: 3, train_loss: 0.6800888776779175, valid_loss: 0.7126150131225586, time: [22.88], best model: 0
Epoch: 4, train_loss: 0.6800166964530945, valid_loss: 0.7125856280326843, time: [23.32], best model: 0
Epoch: 5, train_loss: 0.6800587177276611, valid_loss: 0.7127432227134705, time: [25.03], best model: 0
Epoch: 6, train_loss: 0.6801396012306213, valid_loss: 0.7125070691108704, time: [26.18], best model: 0
Epoch: 7, train_loss: 0.6800600290298462, valid_loss: 0.7125433683395386, time: [27.15], best model: 0
Epoch: 8, train_loss: 0.6800187230110168, valid_loss: 0.7129371762275696, time: [24.79], best model: 0
Epoch: 9, train_loss: 0.6799517869949341, valid_loss: 0.7124120593070984,

In [111]:
K = 3
back_length = 3
Clamp_A = False
sgclstm = SpectralGraphConvolutionalLSTM(K, torch.Tensor(A), A.shape[0], Clamp_A=Clamp_A, output_last = True)
sgclstm, sgclstm_loss = TrainModel(sgclstm, train_dataloader, valid_dataloader, num_epochs = 200)
sgclstm_test = TestModel(sgclstm, test_dataloader, max_speed )

In [129]:
K = 3
back_length = 3
Clamp_A = False
gclstm = GraphConvolutionalLSTM(K, torch.Tensor(A), FFR[back_length], A.shape[0], Clamp_A=Clamp_A, output_last = True)
gclstm, gclstm_loss = TrainModel(gclstm, train_dataloader, valid_dataloader, num_epochs = 200)
gclstm_test = TestModel(gclstm, test_dataloader, max_speed )

Epoch: 0, train_loss: 0.13212086260318756, valid_loss: 0.13522964715957642, time: [10.59], best model: 1
Epoch: 1, train_loss: 0.041993398219347, valid_loss: 0.0428180918097496, time: [11.53], best model: 1
Epoch: 2, train_loss: 0.027062859386205673, valid_loss: 0.027360010892152786, time: [12.14], best model: 1
Epoch: 3, train_loss: 0.019733069464564323, valid_loss: 0.020925279706716537, time: [12.46], best model: 1
Epoch: 4, train_loss: 0.01488890964537859, valid_loss: 0.016917500644922256, time: [13.81], best model: 1
Epoch: 5, train_loss: 0.012853049673140049, valid_loss: 0.01569673977792263, time: [13.48], best model: 1
Epoch: 6, train_loss: 0.010094859637320042, valid_loss: 0.013394540175795555, time: [12.19], best model: 1
Epoch: 7, train_loss: 0.009070070460438728, valid_loss: 0.01281294971704483, time: [12.54], best model: 1
Epoch: 8, train_loss: 0.00839305017143488, valid_loss: 0.012489289976656437, time: [12.91], best model: 1
Epoch: 9, train_loss: 0.007891490124166012, vali